In [1]:
import math
import torch 
import matplotlib 
from matplotlib import pyplot as plt
from torchvision import transforms, utils
import numpy as np
from numpy.linalg import inv
import torchvision
import torchvision.datasets as datasets
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
mnist_testset =datasets.MNIST(root='./data', train=False, download=True, transform=None)
train_x=mnist_trainset.data[0:6000].view(-1,28*28)

def kernel(a, b, param):
    sqdist = (a**2) + (b**2).t() - 2*torch.mm(a, b.t())
    #print(np.sum(b**2,1))
    return torch.exp(-.5 * (1/param**2) * sqdist)
sigma=1
param=.1
#K_ss = kernel(Xtest, Xtest, param,sigma)
#L = np.linalg.cholesky(K_ss + 1e-10*np.eye(n))

images= torch.ones([60000,64],dtype=torch.float32)
images_test=torch.ones([10000,64],dtype=torch.float32)

for i in range(60000):
	trans=torchvision.transforms.ToPILImage(mode=None)
	resize = transforms.Resize(size=(8, 8))
	img=resize(trans(mnist_trainset.data[i]))
	tens=torchvision.transforms.ToTensor()
	img=tens(img).view(-1,8*8)
	images[i]= img
for i in range(10000):
	trans=torchvision.transforms.ToPILImage(mode=None)
	resize = transforms.Resize(size=(8, 8))
	img=resize(trans(mnist_testset.data[i]))
	tens=torchvision.transforms.ToTensor()
	img=tens(img).view(-1,8*8)
	images_test[i]= img

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 22465287.12it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 374128.11it/s]
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 5657648.30it/s]                           
8192it [00:00, 139461.71it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
def kernel2d(data1,data2,param):
  matrix=np.zeros((np.size(data1,0),np.size(data2,0)))
  for i in range(np.size(data1,0)):
    for j in range(np.size(data2,0)):
      ecu=np.sqrt(np.sum((data1[i]-data2[j])**2))
      matrix[i,j]=ecu
  return np.exp(-64*matrix / (2.0 * param**2))

In [0]:
def matrix_cholesky(A):
    L = torch.zeros_like(A)

    for i in range(A.shape[-1]):
        for j in range(i+1):
            s = 0.0
            for k in range(j):
                s = s + L[i,k] * L[j,k]

            L[i,j] = torch.sqrt(A[i,i] - s) if (i == j) else \
                      (1.0 / L[j,j] * (A[i,j] - s))
    return L

In [0]:
def binar(arr,value):
  arr[np.where(arr!=value)]=-4
  arr[np.where(arr==value)]=-2
  return arr+3

In [0]:
def loglikelhood(targets,f):
  Pygivenf=-np.log(1+np.exp(np.multiply(-targets,f)))
  return Pygivenf

In [0]:
def Fderivative(targets,f):
  firstderivative=((targets+1)/2)-loglikelhood(np.ones(np.size(targets)),f)
  return firstderivative

In [0]:
def Sderivative(targets,f):
  secondderivative=np.multiply(np.identity(np.size(targets)),-loglikelhood(np.ones(np.size(targets)),f)*(1-loglikelhood(np.ones(np.size(targets)),f)))
  return secondderivative

In [0]:
def sigma(f):
  sigma=1/(1+np.exp(-f))
  return sigma
  

In [0]:
def predict(targets,ks,fnew):
  fstar=np.dot(ks,Fderivative(targets,fnew))
  return fstar

In [0]:
def predict2(kernel,ks,fnew):
  fstar=np.dot(ks,np.dot(inv(kernel),fnew))
  return fstar

In [0]:
#def integration(fstar,expec,var):
#  sumed=np.zeros(np.size(fsatr))
#  for i in range()
#  sumed=np.dot(sigma(fstar),np.random.multivariate_normal(expec, var, np.size(fsatr))

In [0]:
def newton(targets,f,kernel,iteratons):
  i=0
  marginal1=np.zeros(np.size(targets))
  condition=True
  while ((i<iteratons)&(condition)):
    firstderivative=Fderivative(targets,f)
    secondderivative=Sderivative(targets,f)
    W=secondderivative
    L=np.array(matrix_cholesky(torch.as_tensor(np.identity(np.size(targets))+np.dot(np.dot(np.sqrt(W),kernel),(np.sqrt(W)))+1e-15*np.eye(np.size(targets)))))
    #print(f)
    b=np.dot(W,f)+firstderivative
    a=b-np.dot(np.dot((np.sqrt(W)),inv(L.T)),(np.dot(np.dot(np.dot(inv(L),(np.sqrt(W))),kernel),b)))
    f=np.dot(kernel,a)
    marginal2=-.5*np.dot(np.dot(f,inv(kernel)),f)+loglikelhood(targets,f)-np.linalg.det(np.identity(np.size(targets))+np.dot(np.dot(np.sqrt(W),kernel),(np.sqrt(W))))
    condition=abs(np.sum(marginal1)-np.sum(marginal2))>.01
    #print(np.sum(marginal2))
    marginal1=marginal2
    i=i+1
  return f

In [16]:
samples=50
lscale=28
fin=np.zeros(samples)
targets=binar(np.array(mnist_trainset.targets[0:samples]),0)
kernel=kernel2d(np.array(train_x[0:samples]),np.array(train_x[0:samples]),lscale)
f=newton(targets,fin,kernel,100)
sig=sigma(f)
print(sig)
print(targets)
ks=kernel2d(np.array(train_x[150:200]),np.array(train_x[0:samples]),lscale)
fstar=predict(targets,ks,f)
Efstar=predict2(kernel,ks,f)
W=Sderivative(targets,f)
L=np.array(matrix_cholesky(torch.as_tensor(np.identity(np.size(targets))+np.dot(np.dot(np.sqrt(W),kernel),(np.sqrt(W)))+1e-15*np.eye(np.size(targets)))))
v=np.dot(inv(L),(np.dot(np.sqrt(W),ks)))
V=kernel2d(np.array(train_x[50:100]),np.array(train_x[50:100]),1)-np.dot(v.T,v)
mul=np.dot(sigma(f),np.random.multivariate_normal(Efstar, V, np.size(f)))
pred=sigma(Efstar)
#print(Efstar)
print(pred)
print(binar(np.array(mnist_trainset.targets[150:200]),0))

[0.63626944 0.79114121 0.63414907 0.65694914 0.6365091  0.63240561
 0.65316683 0.6344887  0.65506403 0.63714467 0.64106322 0.64222599
 0.630664   0.63207148 0.66512186 0.63727295 0.63354057 0.63730144
 0.63866172 0.64265851 0.63577355 0.78854903 0.63980386 0.6526541
 0.63803987 0.630605   0.64454117 0.63289959 0.63210167 0.64329167
 0.63576238 0.63733969 0.63900433 0.64148999 0.7939876  0.64149442
 0.64025988 0.78869387 0.63485396 0.64082822 0.65807605 0.63644398
 0.65261097 0.64652967 0.64102703 0.63758266 0.63318568 0.67628261
 0.64170381 0.63399297]
[-1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1]
[0.52856437 0.52627209 0.55109975 0.54203181 0.52762217 0.52588238
 0.52418839 0.52567433 0.52318018 0.51737415 0.53242709 0.51633543
 0.53479255 0.52934862 0.52719565 0.5330429  0.53267946 0.52621378
 0.536383   0.52778313 0.52807461 0.52110836 0.52435498 0.5395275
 0.57095571 0.535230